А. Панасюк БКЛ 182

In [1]:
import sys  
sys.path.insert(0, 'C:/Users/mjo/prozhitotools/')
from prozhitotools import dump
dw = dump.Wrapper(r'C:\Users\mjo\Downloads\prozhito')

In [2]:
import re
import pandas as pd

In [ ]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', -1)

#### Тз

Предложите подход к **оценке** и **анализу** полученной разметки. Чего вы **ждёте от неё**, какие машина допустила **ошибки**, как их может показаться можным **исправить**. ***Почему современный NER допускает такие ошибки?*** Что вы вообще думаете об этом и смежном?

##### Выборка

Я беру записи из дневников за Рождество и Новый год. В имперской России я беру даты с 25 декабря по 1 января, в советской и постсоветской России я беру 31 декабря, 1 января и 7 января (иначе слишком много записей).

Смотрела тексты и выстраивала гипотезы я по полным выборкам за это время (70 текстов и 243), в выборку для разметки я брала около 12 текстов для каждого из периодов, чтобы было удобно отсматривать вывод.

Для конечного теста я брала только тексты, написанные в Российской империи, так как оставалось мало времени.

Также в Новом году и Рождестве часто подводят итоги года и пишут о чем-то кульминационном и важном, упоминают важные события (вот в 1992 году на НГ многие писали о поздравлении Задорнова). Ну и в принципе ощущение, что на эти зимние праздники многое происходит и можно вытащить что-то неочевидное (например, для меня отдаленной не до конца очевидны религиозные употребления Оно, Провидение с большой буквы).

#### Годы, которые мне показались интересными для проверки

1) Восстание декабристов 1825 (НГ 1826)

2) 1856 Крымская война

3) 1907 год революции

4) 1946 год конца войны

5) 1980 Афганская война

6) 1992 первый новый год в РСФСР

10) новый год 2010 (что-то из совсем современного)

#### Гипотезы

1) Ожидаю, что тэггеры сломаются о дневниковые сокращения (А. В., гн., кн., m-me и т.д.)

**NB** В Прожито есть разметка типа М(арья) И(оановна), так вот, я решила скобки не раскрывать, а удалить такие пояснения для чистоты эксперимента. Потому что при работе с необработанными документами это будет проблемой.

Очевидно, можно прописать какие-то правила, но как бы все это заработало? Правила или нейросеть? Мой вывод в конце тетради.

С неименными сокращениями же проще, нужно составить словарь. (гн - господин и т.д.)

2) Тэггеры будут ломаться на иностранных вставках (французский для 19 века, немецкий для 20 века, остальные языки при встрече)

Гипотеза: в наше время на ин. языках пишут скорее названия брендов, а тогда были и NE, и вставки в речи (иногда с большой буквы), непонятно, как объяснить, где разница.

3) Неожиданные для современного исследователя реалии времени, которые по какой-то причине похожи на NE (как Провидение и т.д.).

00) но и в современных текстах есть много проблем, особенно в блогах, в которых пишут как угодно и очень большая вариативность

##### Предобработка

##### 1. Империя

In [134]:
empire = [1826, 1856, 1907]
ussr_ru = [1946, 1980, 1992, 2010]

In [165]:
empire_df = pd.DataFrame(columns=['author', 'date', 'text'])

In [90]:
pattern = re.compile('\[.+?\]')
pattern_encl = re.compile('\([А-яA-z]+?\)')
pattern_link = re.compile('<.+?>')

In [166]:
for year in empire:
    n_year = dw.notes[(year-1, 12, 25) : (year, 1, 1)]
    for it in n_year:
        diary_text = pattern.sub('', it.text)
        diary_text = pattern_link.sub('', diary_text)
        diary_text = pattern_encl.sub('', diary_text)
        diary_text = diary_text.replace('*', '')
        empire_df = empire_df.append({'author': it.author, 'date': it.date, 'text': diary_text}, ignore_index=True)

In [167]:
len(empire_df)  # всего в выборке 70 текстов для имперской России

70

##### 20-21 век

In [135]:
ussr_rudf = pd.DataFrame(columns=['author', 'date', 'text'])

In [136]:
for year in ussr_ru:
    n_year = dw.notes[(year-1, 12, 31) : (year, 1, 1)]
    print(year)
    print(len(n_year))
    for it in n_year:
        diary_text = pattern.sub('', it.text)
        diary_text = pattern_link.sub('', diary_text)
        diary_text = pattern_encl.sub('', diary_text)
        diary_text = diary_text.replace('*', '')
        ussr_rudf = ussr_rudf.append({'author': it.author, 'date': it.date, 'text': diary_text}, ignore_index=True)
    n_year = dw.notes[(year, 1, 7):(year, 1, 7)]
    print(len(n_year))
    for it in n_year:
        diary_text = pattern.sub('', it.text)
        diary_text = pattern_link.sub('', diary_text)
        diary_text = pattern_encl.sub('', diary_text)
        diary_text = diary_text.replace('*', '')
        ussr_rudf = ussr_rudf.append({'author': it.author, 'date': it.date, 'text': diary_text}, ignore_index=True)

1946
39
12
1980
37
9
1992
10
8
2010
10
4


То, что мало текстов после 1950-го, связано с тем, что дневники того времени еще не передали в Прожито

In [143]:
len(ussr_rudf)

129

In [175]:
us_ru = ussr_rudf.text  # сокращаю количество текстов для разметки, потому что все ломается, когда слишком много, 
ur_texts = '\n\n\n'.join(us_ru[::9])  # и непонятно, как анализировать

In [178]:
emptexts = empire_df.text
emptext = '\n\n\n'.join(emptexts[::10])

#### NER

Про NE я знала, что есть Наташа, поэтому мне нужен был рисерч

Сравнение теггеров: 
https://habr.com/ru/post/502366/

Stanza

Natasha

DeepPavlov (здравствуйте Алиса и Алекса)

deepmipt/ner

Polyglot

PullEnti

##### И еще

куча всего с БЕРТом (для которого я маленькая собачка и его сразу опускаю)
http://docs.deeppavlov.ai/en/master/features/pretrained_vectors.html

И еще:

nltk
ner-ru https://github.com/zamgi/lingvo--Ner-ru

##### Что я беру

В 2016 стал лучшим RuEval **PullEnti** https://github.com/dialogue-evaluation/factRuEval-2016

Так что брать буду его. И буду его сравнивать с наташей.

Потому что pullenti -- rule-based, наташа -- эмбеддинги. А мы ожидаем, что правила на старых текстах не работают.

In [151]:
from pullenti_wrapper.processor import (
    Processor,
    MONEY,
    URI,
    PHONE,
    DATE,
    KEYWORD,
    DEFINITION,
    DENOMINATION,
    MEASURE,
    BANK,
    GEO,
    ADDRESS,
    ORGANIZATION,
    PERSON,
    MAIL,
    TRANSPORT,
    DECREE,
    INSTRUMENT,
    TITLEPAGE,
    BOOKLINK,
    BUSINESS,
    NAMEDENTITY,
    WEAPON,
)

In [186]:
result = processor(emptext)

In [197]:
matches = []
if result.matches != []:
    for match in result.matches:
        matches.append(match)

In [270]:
matches = result.matches

In [268]:
emppul = pd.DataFrame(columns=['label', 'slots'])  # пустой дф чтобы туда записать найденные мэтчи

In [271]:
len(matches)  # их 78

78

In [272]:
for match in matches:
    label = match.referent.label
    emppul = emppul.append({'label':label, 'slots':match.referent.slots}, ignore_index=True)

In [273]:
emppul  # записываю я очень непорядочно, но так хотя бы видно. Я не знаю, сколько может быть в теории слотов, так что пусть

,label,slots
0,PERSON,"[Slot(key='SEX', value='MALE'), Slot(key='LASTNAME', value='САНИН'), Slot(key='FIRSTNAME', value='АЛЕКСАНДР')]"
1,PERSON,"[Slot(key='SEX', value='MALE'), Slot(key='LASTNAME', value='САНИН'), Slot(key='FIRSTNAME', value='АЛЕКСАНДР')]"
2,PERSON,"[Slot(key='SEX', value='MALE'), Slot(key='LASTNAME', value='САНИН'), Slot(key='FIRSTNAME', value='АЛЕКСАНДР')]"
3,GEO,"[Slot(key='NAME', value='ВЕНА'), Slot(key='TYPE', value='город')]"
4,GEO,"[Slot(key='ALPHA2', value='RU'), Slot(key='NAME', value='РФ'), Slot(key='NAME', value='РОССИЙСКАЯ ФЕДЕРАЦИЯ'), Slot(key='NAME', value='РОССИЯ'), Slot(key='TYPE', value='государство')]"
5,GEO,"[Slot(key='ALPHA2', value='RU'), Slot(key='NAME', value='РФ'), Slot(key='NAME', value='РОССИЙСКАЯ ФЕДЕРАЦИЯ'), Slot(key='NAME', value='РОССИЯ'), Slot(key='TYPE', value='государство')]"
6,GEO,"[Slot(key='ALPHA2', value='TR'), Slot(key='NAME', value='ТУРЕЦКАЯ РЕСПУБЛИКА'), Slot(key='NAME', value='ТУРЦИЯ'), Slot(key='TYPE', value='государство')]"
7,PERSON,"[Slot(key='SEX', value='MALE'), Slot(key='LASTNAME', value='САНИН'), Slot(key='FIRSTNAME', value='АЛЕКСАНДР')]"
8,GEO,"[Slot(key='ALPHA2', value='CN'), Slot(key='NAME', value='КНР'), Slot(key='NAME', value='КИТАЙСКАЯ НАРОДНАЯ РЕСПУБЛИКА'), Slot(key='NAME', value='КИТАЙ'), Slot(key='NAME', value='ПОДНЕБЕСНАЯ'), Slot(key='TYPE', value='государство')]"
9,GEO,"[Slot(key='ALPHA2', value='RU'), Slot(key='NAME', value='РФ'), Slot(key='NAME', value='РОССИЙСКАЯ ФЕДЕРАЦИЯ'), Slot(key='NAME', value='РОССИЯ'), Slot(key='TYPE', value='государство')]"


##### Вывод pullenti

Видно, что хорошо достает названия городов, стран и понятные фамилии (а вот имя императора Александра почему-то не достал и объединил его с фамилией Санин. Потому что в конце один раз упоминается Александр Санин). Также разобрал "министерство двора", т.к. это вполне себе могущая входить в rules вхождение.

Не разобрался и с сокращениями.

Не достал имя императора Павла.

Не достал иностранные имена, хотя мог бы. "Вечер у мудреца Royer-Collard". ( а вот Наташа подумала, что это организация, но хотя бы подумала). Не разобралси НИ С КАКИМИ иностранным именами. Наташа хотя бы некоторые поняла.

Не достал названия "Гулльский инцидент", название город Гулль. "Русь" написал организацией, а не geo, т.к. есть Банк Русь. Т.е. слишком много правил под настоящие реалии.

##### В общем, в старых текстах не до правил

In [267]:
result

Result(
    text='1 час пополудни. Сию минуту возвратился из русской церкви: мы служили панихиду по незабвенном императоре, — и возносили на литургии имя нового! Почти все русские были в церкви. Трудно выразить впечатление, которое произвела на меня молитва о усопшем! Я озирался вокруг себя и видел памятники, живые, царствования Александра, из коих некоторые принадлежат истории, другие останутся в записках двора или по крайней мере в анекдотах его. Посол, коего политическое бытие сотворил Александр всемогуществом своего сана — и стечением необыкновенных обстоятельств; за ним памятник пяти царствований — светлейший Разумовский, еще бодрящийся, но уже не бодрствующий у чужих престолов, обязанный Александру настоящим благосостоянием своим, утратив прежнее с обгорелыми развалинами великолепного дворце своего в Вене. Далее Чичагов, непримиримый враг русского правительства, смелый говорун в собрании министров и Совета, мечтающий иметь право злословить Россию в лице ее правителей и привязавши

### Наташа

In [259]:
from navec import Navec
from slovnet import NER
from ipymarkup import show_span_ascii_markup as show_markup

navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
ner = NER.load('slovnet_ner_news_v1.tar')
ner.navec(navec)

markup = ner(emptext)

show_markup(markup.text, markup.spans)

1 час пополудни. Сию минуту возвратился из русской церкви: мы служили 
панихиду по незабвенном императоре, — и возносили на литургии имя 
нового! Почти все русские были в церкви. Трудно выразить впечатление, 
которое произвела на меня молитва о усопшем! Я озирался вокруг себя и 
видел памятники, живые, царствования Александра, из коих некоторые 
                                     PER───────                    
принадлежат истории, другие останутся в записках двора или по крайней 
мере в анекдотах его. Посол, коего политическое бытие сотворил 
Александр всемогуществом своего сана — и стечением необыкновенных 
PER──────                                                         
обстоятельств; за ним памятник пяти царствований — светлейший 
Разумовский, еще бодрящийся, но уже не бодрствующий у чужих престолов,
PER────────                                                           
 обязанный Александру настоящим благосостоянием своим, утратив прежнее
           PER───────                  

In [262]:
doc_ner = ner(emptext)

In [264]:
len(doc_ner.spans)

136

Наташа молодец, даже с сокращениями разобралась. И Ал, и Кн Тюфякин были пойманы.

Иностранщиу на нерусском впрочем тоже не везде поймала.

Не везде выделила города (Паловск), имена (Георгию). Очевидно, дело в том, что они 1-ые в строке и с большой буквы, и Наташа списала это на начало строки (А потом, когда Павловск не в начале строки, нашла).

Из-за этого же выделила Мама как NE, хотя Мама это родственная мама.

Не выделила Иоанна Грозного в кавычках :( pullenti выделил. Очевидно, потому, что когда-то наташе сказали, что не все, что в кавычках -- имя собственное, а может быть например (цитата), и склонилось к этому варианту.

**Но в целом разбор вышел вполне приличным**

### Вывод

Наташа (точнее, словнет) с эмбеддингами (новостными) справилась лучше, чем более rule-based pullenti на старом тексте. 136 NE vs 78 (притом, что не было подсчета дат у наташи).

Предложите подход к оценке и анализу полученной разметки. Чего вы ждёте от неё, какие машина допустила ошибки, как их может показаться можным исправить. Почему современный NER допускает такие ошибки? Что вы вообще думаете об этом и смежном?

Почему так? Наташа больше настроена на "догадывание", а не на правила. И для более старого материала этот подход выграл.

Возможно, стоит к гибкости эмбеддингов добавить еще правил, которые бы описывали особенности письма того времени, и тогда было бы шикарно. Или было бы еще лучше обучить модель на разметке текстов того времени. И, например, объяснить, что вот такие сочетания нерусскоязычных символов в тексте тоже могли быть именами.

Как подходить к анализу? Разбивать на классы и смотреть, где кому что удалось сделать, а где нет. Оценивать так же и брать от подходов лучшее.